In [29]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [30]:
offers_connection = sqlite3.connect('../datasets/offers_dataset.db')
offers_frame = pd.read_sql_query('''SELECT * FROM offers''', offers_connection)
offers_frame.set_index('ID', inplace=True)
offers_connection.close()

# [!] Le descrizioni all'interno del notebook non sono mostrate nella loro versione completa
#     per accedere ad una specifica descrizione usare il metodo: offers_frame.loc[ID, NomeColonna]
offers_frame

,Name,Description,Location
ID,,,
1,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria"
2,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona
3,Software Developer,We are looking for talented and passionate peo...,55100 Lucca
4,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata"
5,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia"
...,...,...,...
850,JUNIOR DEVELOPER,Sede lavoro: Bergamo | Tempo pieno \n Livello:...,"Bergamo, Lombardia"
851,Sviluppatore Java Junior,TRIA ricerca uno sviluppatore Java JuniorNeola...,None
852,Stage Software Developer,CHI SIAMO \n Golilla è la start up delle azi...,20089 Rozzano


## Data Exploration
Come prima cosa esaminiamo il nostro dataset di offerte, in particolare dovremmo verificare la qualità dei dati:
- Base   
    - Presenza di valori nulli
    - (**Importante**) Presenza di sinonimi geografici
    - Presenza di informazioni utili
- Avanzato
    - Analisi delle Parole Chiave (es. Titoli)
    - Analisi della distribuzione dei dati

**Presenza di Valori Nulli e Duplicati**
Come è possibile vedere di seguito il dataset contiene valori nulli solo per la colonna Location, il che però è normale, in quanto non tutte le Offerte di lavoro sono in presenza, quindi le Location mancanti rappresentano il numero di Offerte di lavoro in Remote.
Questa informazione è importante perchè in seguito ci sarà la necessità di rappresentare queste informazioni in modo omogeneo, in modo tale che un modello il cui scopo è raggruppare le Offerte (clustering, come proposto nell'Analisi del Problema) possa farlo in modo adeguato.

In [31]:
offers_frame.drop_duplicates(inplace=True)
offers_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1 to 101
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         50 non-null     object
 1   Description  50 non-null     object
 2   Location     38 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


## Presenza di Sinonimi Geografici
Il dataset contiene attualmente molteplici istanze in cui la stessa Location è riportata in modi diversi, come \<Cap>, \<Città> e \<Città>, \<Regione>.


In [32]:
#example_city = "Bologna"

#onsite_offers = offers_frame[offers_frame["Location"].notna()]
#geo_syn = onsite_offers.Location.map(lambda text: example_city in text)
#onsite_offers[geo_syn]

**Analisi dei Titoli**
Di seguito vengono analizzati i Titoli delle Offerte di lavoro con lo scopo di verificare la presenza di termini comuni, in particolare vengono usate delle basilari tecniche di Natural Language Processing come:
- **Tokenization**, utilizzata per dividere frasi o documenti in parole (*title.split()*) 
- **Rimozione della Punteggiatura**, utilizzata per migliorare la qualità dei dati estratti, in modo da non influenzare i risultati 
- **Normalizzazione**, utilizzata per ridurre la complessità del vocabolario e trattare parole con maiuscole e minuscole come equivalenti (*t.lower()*)

In [33]:
from string import punctuation
from collections import Counter

def count_words(document: str, counter: Counter):
    """
    :param document: a string representation of the document to inspect
    :param counter: a Python built-in object used to count occurrences of items
    """
    punct = [p for p in punctuation]
    for t in document.split():
        if not t in punct:
            counter[t.lower()] += 1


titles_counts = Counter()
offers_frame.Name.map(lambda text: count_words(text, titles_counts))

titles_counts

Counter({'software': 36,
         'developer': 30,
         'engineer': 8,
         'junior': 6,
         'senior': 4,
         'sviluppatore': 3,
         'java': 3,
         'programmatore': 3,
         'c#': 3,
         'graduate': 3,
         'frontend': 2,
         'settore': 2,
         'sanità': 2,
         'full': 2,
         'engineering': 2,
         'apprenticeship': 2,
         '2024': 2,
         'intern': 2,
         'ibp': 1,
         'algorithms': 1,
         'development': 1,
         'unity': 1,
         'game': 1,
         'stage': 1,
         'web': 1,
         'ai': 1,
         'trainer': 1,
         '(contract)': 1,
         'tecnico': 1,
         'bilance': 1,
         'digitali': 1,
         'supermercati': 1,
         'asp.net': 1,
         'manufacturing': 1,
         'relocation': 1,
         'usa': 1,
         'front-end': 1,
         'app': 1,
         'programmer': 1,
         'hmi': 1,
         'flutter': 1,
         'remote': 1,
         'stack': 1,
    

Analisi delle occorrenze di parole:

In [34]:
# Jacopo plotta qui

### Skill Extraction
Per procedere con l'estrazione delle Competenze dalle Offerte per prima cosa importiamo il relativo dataset.   

In [35]:
with sqlite3.connect('../datasets/skills_dataset.db') as skills_connection:
    skills_frame = pd.read_sql_query('SELECT * FROM skills', skills_connection)
    skills_frame.set_index('ID', inplace=True)
    
skills_frame

,SKILL,TYPE
ID,,
0,JavaScript,Programming Language
1,HTML,Programming Language
2,Python,Programming Language
3,SQL,Programming Language
4,TypeScript,Programming Language
...,...,...
95,APT,Tool
96,Unity 3D,Tool
97,Pacman,Tool


In [36]:
# Get Skill list
skills_list = [skill.lower() for skill in skills_frame['SKILL'].tolist()]

# Symbols to remove
punct = [p for p in punctuation]
punct.remove('+')
punct.remove('#')

removal = {p: '' for p in punct}
removal['\n'] = ''

def remove_symbols(description: str, remove_map: dict) -> str:
    for old, new in remove_map.items():
        description = description.replace(old, new)
    return description.lower()


def extract_skills(description: str, available_skills: list) -> set:
    s = set()
    for word in description.split():
        if word in available_skills:
            s.add(word)
    return s 

In [37]:
for i, offer_description in enumerate(offers_frame.loc[:, 'Description']):
    desc = remove_symbols(offer_description, removal)
    offer_skills = extract_skills(desc, skills_list)
    if not offer_skills:
        print(f'{i}-Offer: No Skills')
    else:
        print(offer_skills)


0-Offer: No Skills
{'c', 'python', 'java', 'c#', 'c++'}
{'java', 'oracle', 'javascript', 'angular'}
{'python', 'c++'}
{'angularjs', 'javascript', 'java', 'c#', 'sql'}
{'java'}
6-Offer: No Skills
7-Offer: No Skills
{'javascript', 'css', 'html'}
{'java', 'mysql'}
{'html', 'jquery', 'javascript', 'c#', 'css', 'sql'}
{'java'}
{'react', 'go', 'javascript', 'java', 'kubernetes', 'docker', 'php'}
13-Offer: No Skills
{'react', 'angular', 'typescript'}
15-Offer: No Skills
16-Offer: No Skills
{'java'}
{'java', 'oracle', 'react', 'angular'}
19-Offer: No Skills
{'java', 'mysql', 'c#', 'python', 'c++', 'sql'}
{'react', 'flutter', 'oracle', 'javascript', 'typescript', 'dart', 'angular'}
{'c#', 'sql'}
{'javascript', 'python', 'sql', 'php'}
24-Offer: No Skills
{'dart', 'flutter'}
{'vite', 'docker', 'c#', 'sql'}
{'java', 'c#', 'php'}
{'python', 'java', 'c#'}
{'java', 'c', 'c++'}
{'django', 'postgresql', 'javascript', 'python', 'mysql', 'css', 'docker', 'sql'}
{'mysql', 'sql'}
32-Offer: No Skills
{'sql'

In [38]:
# Package necessari per la manipolazione di descrizioni in italiano (es. TF-IDF)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('italian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows10\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## Data Preparation
Per permettere il completo soddisfacimento dei requisiti, alcuni dati mancanti alle offerte dovranno essere aggiunti al dataset, tra cui:
- LocationType: il tipo di Location, se in remoto (Remote) o in sede (OnSite)
- Languages: le lingue naturali necessarie per l'offerta di lavoro

### Tipo di Location
Per estrarre il tipo di Location dalle informazioni attualmente già presenti nel dataset, 
saranno interpretate come di tipo Remote tutte le offerte che non presentano una Location.

In [39]:
LOCATION_TYPES = {"Remote" : "Remote", "On Site" : "OnSite"}
locations_present = offers_frame["Location"].notna()
types = []

for is_present in locations_present:
    if is_present:
        types.append(LOCATION_TYPES["On Site"])
    else:
        types.append(LOCATION_TYPES["Remote"])

offers_frame.loc[:, "LocationType"] = types

offers_frame

,Name,Description,Location,LocationType
ID,,,,
1,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria",OnSite
2,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona,OnSite
3,Software Developer,We are looking for talented and passionate peo...,55100 Lucca,OnSite
4,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata",OnSite
5,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia",OnSite
6,SOFTWARE DEVELOPER,La passione ci guida in tutto ciò che facciamo...,60030 Monsano,OnSite
7,IBP Junior Algorithms Software Development,Pirelli is looking for the following profile t...,"Bari, Puglia",OnSite
8,Unity game developer,Junior or SeniorWork from home! Literally alw...,None,Remote
9,Frontend,Esperienza richiesta : 3-10 anniConoscenze ric...,None,Remote
